<a href="https://colab.research.google.com/github/TheGodseeker/ML_PR/blob/main/lab3/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание по практике 3

##Обработка пропущенных значений

Обработайте пропущенные значения:


* Удалите пропущенные значения
* Заполните при помощи медианы, среднего для числовых значений
* Заполните при помощи моды для категориальных значений
* Интерполируйте
Выведите статистку после каждого метода и объясните


In [1]:
!pip install scikit-learn

In [8]:
import pandas as pd



df = pd.read_csv('/content/anime.csv')


df.describe(include="all")

,anime_id,name,genre,type,episodes,rating,members
count,12294.000000,12294,12232,12269,12294,12064.000000,1.229400e+04
unique,NaN,12292,3264,6,187,NaN,NaN
top,NaN,Shi Wan Ge Leng Xiaohua,Hentai,TV,1,NaN,NaN
freq,NaN,2,823,3787,5677,NaN,NaN
mean,14058.221653,NaN,NaN,NaN,NaN,6.473902,1.807134e+04
std,11455.294701,NaN,NaN,NaN,NaN,1.026746,5.482068e+04
min,1.000000,NaN,NaN,NaN,NaN,1.670000,5.000000e+00
25%,3484.250000,NaN,NaN,NaN,NaN,5.880000,2.250000e+02
50%,10260.500000,NaN,NaN,NaN,NaN,6.570000,1.550000e+03
75%,24794.500000,NaN,NaN,NaN,NaN,7.180000,9.437000e+03


In [7]:
#Удаление
df_del = df.dropna()

#df_del
df_del.describe(include="all")

,anime_id,name,genre,type,episodes,rating,members
count,12017.000000,12017,12017,12017,12017,12017.000000,1.201700e+04
unique,NaN,12015,3229,6,187,NaN,NaN
top,NaN,Shi Wan Ge Leng Xiaohua,Hentai,TV,1,NaN,NaN
freq,NaN,2,816,3668,5571,NaN,NaN
mean,13638.001165,NaN,NaN,NaN,NaN,6.478264,1.834888e+04
std,11231.076675,NaN,NaN,NaN,NaN,1.023857,5.537250e+04
min,1.000000,NaN,NaN,NaN,NaN,1.670000,1.200000e+01
25%,3391.000000,NaN,NaN,NaN,NaN,5.890000,2.250000e+02
50%,9959.000000,NaN,NaN,NaN,NaN,6.570000,1.552000e+03
75%,23729.000000,NaN,NaN,NaN,NaN,7.180000,9.588000e+03


In [10]:
#Заполнение при помощи медианы

df_med = df.fillna(df.median())
df_med.describe(include="all")

<ipython-input-10-b760a8cef12d>:3: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_med = df.fillna(df.median())


,anime_id,name,genre,type,episodes,rating,members
count,12294.000000,12294,12232,12269,12294,12294.000000,1.229400e+04
unique,NaN,12292,3264,6,187,NaN,NaN
top,NaN,Shi Wan Ge Leng Xiaohua,Hentai,TV,1,NaN,NaN
freq,NaN,2,823,3787,5677,NaN,NaN
mean,14058.221653,NaN,NaN,NaN,NaN,6.475700,1.807134e+04
std,11455.294701,NaN,NaN,NaN,NaN,1.017179,5.482068e+04
min,1.000000,NaN,NaN,NaN,NaN,1.670000,5.000000e+00
25%,3484.250000,NaN,NaN,NaN,NaN,5.900000,2.250000e+02
50%,10260.500000,NaN,NaN,NaN,NaN,6.570000,1.550000e+03
75%,24794.500000,NaN,NaN,NaN,NaN,7.170000,9.437000e+03


In [11]:
#Заполнение при помощи среднего

df_mean = df.fillna(df.mean())
df_mean.describe(include="all")

<ipython-input-11-b32b3f98325b>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_mean = df.fillna(df.mean())


,anime_id,name,genre,type,episodes,rating,members
count,12294.000000,12294,12232,12269,12294,12294.000000,1.229400e+04
unique,NaN,12292,3264,6,187,NaN,NaN
top,NaN,Shi Wan Ge Leng Xiaohua,Hentai,TV,1,NaN,NaN
freq,NaN,2,823,3787,5677,NaN,NaN
mean,14058.221653,NaN,NaN,NaN,NaN,6.473902,1.807134e+04
std,11455.294701,NaN,NaN,NaN,NaN,1.017096,5.482068e+04
min,1.000000,NaN,NaN,NaN,NaN,1.670000,5.000000e+00
25%,3484.250000,NaN,NaN,NaN,NaN,5.900000,2.250000e+02
50%,10260.500000,NaN,NaN,NaN,NaN,6.550000,1.550000e+03
75%,24794.500000,NaN,NaN,NaN,NaN,7.170000,9.437000e+03


In [12]:
#Заполнение при помощи моды

df_mode = df.fillna(df["genre"].mode()[0])
df.fillna(df["type"].mode()[0])

df_mode.describe(include="all")

,anime_id,name,genre,type,episodes,rating,members
count,12294.000000,12294,12294,12294,12294,12294,1.229400e+04
unique,NaN,12292,3264,7,187,599,NaN
top,NaN,Shi Wan Ge Leng Xiaohua,Hentai,TV,1,Hentai,NaN
freq,NaN,2,885,3787,5677,230,NaN
mean,14058.221653,NaN,NaN,NaN,NaN,NaN,1.807134e+04
std,11455.294701,NaN,NaN,NaN,NaN,NaN,5.482068e+04
min,1.000000,NaN,NaN,NaN,NaN,NaN,5.000000e+00
25%,3484.250000,NaN,NaN,NaN,NaN,NaN,2.250000e+02
50%,10260.500000,NaN,NaN,NaN,NaN,NaN,1.550000e+03
75%,24794.500000,NaN,NaN,NaN,NaN,NaN,9.437000e+03


In [15]:
#Заполнение при помощи интерполяции

 df_inter=df.interpolate() #я без понятия почему компилятор бесится именно на это объявление

 df_inter.describe(include="all")

IndentationError: ignored

**Объясните**

Самый эффективный метод - удаление пустых переменных, но он способен отсечь вписывающиеся в статистику данные. Медианна и средняя тоже могут помочь, но их ипользование способно исказить статистику.

##Поиск выбросов и аномалий

**Те, кто выполняет задания со * делают поиск выбросов с помощью написанного ранее класса**

Найти выбросы по числовой переменной (использовать метод основанный на среднем и на межквартильном размахе). Объясните результат

Вывести статистику с выбросами и без. Сравните и объясните результат


In [17]:
class BurstDestr:

    def __init__(self, df):
      self.df = df

    def clean_with_mean(self, column_name):
          mean_data = mean(self.df[column_name])
          maximum = mean_data * 2
          result = []

          for item in self.df[column_name]:
            if abs(item - mean_data) < maximum:
                result.append(item)

          return result

      def

In [ ]:
#Методом среднего

In [ ]:
#При помощи межквартильного размаха

In [ ]:
#Статистика и объяснение

**Объяснения**

##Дубликаты

Проверьте датафрейм на дубликаты и удалите их, если дублей нет - создайте.

In [25]:
#Проверка на дубли
dupls = df.duplicated()

for i in dupls:
  if (i): print(i)

In [28]:
#Дублей нет, нужно создать

df_with_dupls = pd.concat((df, df.loc[0:100]))
df_with_dupls

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
96,9756,Mahou Shoujo Madoka★Magica,"Drama, Magic, Psychological, Thriller",TV,12,8.51,462974
97,10165,Nichijou,"Comedy, School, Slice of Life",TV,26,8.52,254436
98,13601,Psycho-Pass,"Action, Police, Psychological, Sci-Fi",TV,22,8.50,509109
99,22789,Barakamon,"Comedy, Slice of Life",TV,12,8.50,225927


In [29]:
#Удаление дублей

df_no_duplicates = df_with_dupls.drop_duplicates()
df_no_duplicates

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


## Преобразование признаков

Масштабируйте числовые признаки через MinMaxScaler, StandardScaler

In [32]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

num_features = df.select_dtypes(include=['float64']).columns

# Creating instances
scaler_minmax = MinMaxScaler()
min_max_scaler = df.copy()
min_max_scaler[num_features] = scaler_minmax.fit_transform(df[num_features])
min_max_scaler.Score.unique() # преобразованные уникальные значения в числовом призанке в диапозоне [0, 1]


standard_scaler = StandardScaler()
st_scaler = df.copy()
st_scaler[num_features] = standard_scaler.fit_transform(df[num_features])
st_scaler.Score.unique() # преобразованные уникальные значения в числовом призанке в диапозоне [0, 1]

AttributeError: ignored

Масштабируйте категориальные признаки через OneHotEncoder

In [36]:
from sklearn.preprocessing import OneHotEncoder

cat_features = df.select_dtypes(include=['category']).columns #пытался вставить свои кат признаки - не работает

# Create an instance of OneHotEncoder
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[cat_features])
encoded_columns = encoder.get_feature_names_out(cat_features)
encoded_columns

array([], dtype=object)

##Задание для гениев*


Реализовать расчет корреляции Спирмана

In [ ]:
#Функция по расчету тут

In [ ]:
#Пример использования тут и объяснение принципа работы

###Пример реализации

In [ ]:
import pandas as pd
import numpy as np


def corr_pearson(column_1, column_2):
  cov = np.cov(column_1, column_2)
  return cov/(column_1.std() * column_2.std())

In [ ]:
data = {
    'Рост': [170, 155, 180, 162, 175],
    'Вес': [65, 50, 75, 55, 70]
}
df = pd.DataFrame(data)
df

,Рост,Вес
0,170,65
1,155,50
2,180,75
3,162,55
4,175,70


In [ ]:
corr_pearson(df["Рост"], df["Вес"])

array([[0.96593129, 0.99674864],
       [0.99674864, 1.03527033]])